In [1]:
import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress Tensorflow verbose prints
warnings.simplefilter(action='ignore', category=FutureWarning)

%load_ext autoreload
%autoreload 2

from form_train_data import load_data
import matplotlib.pyplot as plt
from scipy import spatial
from pathlib import Path
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
import itertools
import random
import math
import cv2

data_path = Path.cwd().parent / 'datasets'
train_dataset_path = data_path / 'CASIA-WebFace-112x96'
test_dataset_path = data_path / 'lfw_112x96'

## Load data

In [2]:
print("CASIA pairs for training")
print("LFW pairs for testing")
casia_pairs, lfw_pairs = load_data(data_path)

print("shuffle casia list")
np.random.shuffle(casia_pairs)

CASIA pairs for training
LFW pairs for testing
Loading train data
Loading test data
shuffle casia list


## Train dataframe

In [3]:
casia_list = casia_pairs.tolist()

# convert to pandas dataframe
train_dataframe = pd.DataFrame(casia_list) # takes long time
train_dataframe['flag'] = train_dataframe['flag'].astype(str)
print("train dataframe")
print(train_dataframe)

del casia_pairs, casia_list

train dataframe
                    fileL            fileR flag
0         0951148/123.jpg  1199107/004.jpg    0
1         1249883/244.jpg  1249883/193.jpg    1
2         1501388/033.jpg  1261997/021.jpg    0
3         0192505/075.jpg  0192505/226.jpg    1
4         0000545/400.jpg  0000545/430.jpg    1
...                   ...              ...  ...
46852987  0000837/082.jpg  0000837/007.jpg    1
46852988  0781981/026.jpg  0718647/033.jpg    0
46852989  1715118/238.jpg  1715118/212.jpg    1
46852990  2093097/113.jpg  0502425/163.jpg    0
46852991  0001334/025.jpg  0001334/020.jpg    1

[46852992 rows x 3 columns]


## Test dataframe

In [4]:
lfw_list = lfw_pairs.tolist()

# convert to pandas dataframe
test_dataframe = pd.DataFrame(lfw_list) # takes long time
test_dataframe['flag'] = test_dataframe['flag'].astype(str)
print("test dataframe")
print(test_dataframe)

del lfw_pairs, lfw_list

test dataframe
                                                  fileL  \
0                    Abel_Pacheco/Abel_Pacheco_0001.jpg   
1                Akhmed_Zakayev/Akhmed_Zakayev_0001.jpg   
2                Akhmed_Zakayev/Akhmed_Zakayev_0002.jpg   
3                  Amber_Tamblyn/Amber_Tamblyn_0001.jpg   
4                Angela_Bassett/Angela_Bassett_0001.jpg   
...                                                 ...   
5556                     Scott_Wolf/Scott_Wolf_0002.jpg   
5557  Sergei_Alexandrovitch_Ordzhonikidze/Sergei_Ale...   
5558                     Shane_Loux/Shane_Loux_0001.jpg   
5559                 Shawn_Marion/Shawn_Marion_0001.jpg   
5560     Slobodan_Milosevic/Slobodan_Milosevic_0002.jpg   

                                       fileR flag  
0         Abel_Pacheco/Abel_Pacheco_0004.jpg    1  
1     Akhmed_Zakayev/Akhmed_Zakayev_0003.jpg    1  
2     Akhmed_Zakayev/Akhmed_Zakayev_0003.jpg    1  
3       Amber_Tamblyn/Amber_Tamblyn_0002.jpg    1  
4     Angela_Bas

## Train generator

In [5]:
# https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
# https://stackoverflow.com/questions/49404993/keras-how-to-use-fit-generator-with-multiple-inputs

def preprocess_image(image):
    image = (image - 127.5) / 128
    return image


def train_generator(train_dataframe, batch_size_):
    class_mode_ = "binary"
    generator = ImageDataGenerator(preprocessing_function=preprocess_image, validation_split=0.0)
    
    train_generator_X1 = generator.flow_from_dataframe(
                             dataframe=train_dataframe,
                             directory=str(train_dataset_path) + "/",
                             x_col="fileL",
                             y_col="flag",
                             subset="training",
                             batch_size=batch_size_,
                             seed=42,
                             shuffle=True,
                             class_mode=class_mode_,
                             color_mode='rgb',
                             target_size=(112, 96))
    
    train_generator_X2 = generator.flow_from_dataframe(
                             dataframe=train_dataframe,
                             directory=str(train_dataset_path) + "/",
                             x_col="fileR",
                             y_col="flag",
                             subset="training",
                             batch_size=batch_size_,
                             seed=42,
                             shuffle=True,
                             class_mode=class_mode_,
                             color_mode='rgb',
                             target_size=(112, 96))
    while True:
        X1i = train_generator_X1.next()
        X2i = train_generator_X2.next()
        yield [X1i[0], X2i[0]], X1i[1]

## Test generator

In [6]:
def test_generator(test_dataframe, batch_size):
    class_mode_ = "binary"
    test_datagen = ImageDataGenerator(
                             preprocessing_function=preprocess_image) 
    test_generator_X1 = test_datagen.flow_from_dataframe(
                             dataframe=test_dataframe,
                             directory=str(test_dataset_path) + "/",
                             x_col="fileL",
                             y_col="flag",
                             batch_size=batch_size,
                             seed=42,
                             shuffle=False,
                             class_mode=class_mode_,
                             color_mode='rgb',
                             target_size=(112,96))

    test_generator_X2 = test_datagen.flow_from_dataframe(
                             dataframe=test_dataframe,
                             directory=str(test_dataset_path) + "/",
                             x_col="fileR",
                             y_col="flag",
                             batch_size=batch_size,
                             seed=42,
                             shuffle=False,
                             class_mode=class_mode_,
                             color_mode='rgb',
                             target_size=(112,96))
    while True:
        X1i = test_generator_X1.next()
        X2i = test_generator_X2.next()
        yield [X1i[0], X2i[0]], X1i[1]

## Model

In [7]:
from tensorflow.keras.layers import Conv2D, Add, Activation, PReLU, Dense, Input, ZeroPadding2D, Lambda, GlobalAveragePooling2D, Dot 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CosineSimilarity
from tensorflow.keras.optimizers import RMSprop
# from keras.engine.topology import Layer
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow import keras

def conv_3_block(input, filters):
    x = ZeroPadding2D(padding=(1, 1))(input)
    x = Conv2D(filters, 3, strides=2, padding='valid', kernel_initializer='glorot_uniform')(x)
    r1 = PReLU()(x)

    x = ZeroPadding2D(padding=(1, 1))(r1)
    x = Conv2D(filters, 3, strides=1, padding='valid', kernel_initializer=TruncatedNormal(stddev=0.01))(x)
    r2 = PReLU()(x)

    x = ZeroPadding2D(padding=(1, 1))(r2)
    x = Conv2D(filters, 3, strides=1, padding='valid', kernel_initializer=TruncatedNormal(stddev=0.01))(x)
    r3 = PReLU()(x)

    x = Add()([r1, r3])
    return x 

def conv_2_block(input, filters):
    x = ZeroPadding2D(padding=(1, 1))(input)
    x = Conv2D(filters, 3, strides=1, padding='valid', kernel_initializer=TruncatedNormal(stddev=0.01))(x)
    x = PReLU()(x)

    x = ZeroPadding2D(padding=(1, 1))(x)
    x = Conv2D(filters, 3, strides=1, padding='valid', kernel_initializer=TruncatedNormal(stddev=0.01))(x)
    x = PReLU()(x)

    x = Add()([input, x])
    return x

def sphereface20(input_shape):
    input = Input(shape=input_shape)
    x = conv_3_block(input, 64)
    x = conv_3_block(x, 128)
    x = conv_2_block(x, 128)
    x = conv_3_block(x, 256)
    x = conv_2_block(x, 256)
    x = conv_2_block(x, 256)
    x = conv_2_block(x, 256)
    x = conv_3_block(x, 512)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, kernel_initializer='glorot_uniform')(x)
    
    model = Model(input, x)
    return model

# https://keras.io/examples/mnist_siamese/
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

# https://stackoverflow.com/questions/51003027/computing-cosine-similarity-between-two-tensors-in-keras
def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)


# network definition
input_shape = (112, 96, 3)
base_network = sphereface20(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

# serialize model to JSON
folder_path = Path.cwd().parent / 'models' / 'sphereface_20_keras'
folder_path.mkdir(parents=True, exist_ok=True)
model_path = str(folder_path / 'sphereface_20.json')
model_json = base_network.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def custom_loss(yTrue,yPred):
    return K.sum(K.log(yTrue) - K.log(yPred))

# def cosine_similarity(y_true, y_pred):
#     y = tf.constant([c1,c2])
#     x = K.l2_normalize(y_true, -1)
#     y = K.l2_normalize(y_pred, -1)
#     s = K.mean(x * y, axis=-1, keepdims=False) * 10
#     return s

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001,
    decay_steps=10000,
    decay_rate=0.96,
    staircase=True)
# optimizer_ = keras.optimizers.SGD(learning_rate=lr_schedule)

# rms_ = RMSprop()
optimizer_ = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-07)
# loss_ = custom_loss
# loss_ = CosineSimilarity(axis=-1, name='cosine_similarity')

model.compile(loss=contrastive_loss, optimizer=optimizer_, metrics=[accuracy])
# model.compile(loss=loss_, optimizer=rms_, metrics=['accuracy'])

In [9]:
batch_size_ = 32
images_per_epoch_ = 100000
epochs_ = 5

# https://github.com/keras-team/keras/issues/10855
hist = model.fit(train_generator(train_dataframe, batch_size_), 
                 steps_per_epoch = images_per_epoch_ // batch_size_,
                 epochs = epochs_,
                 validation_data = test_generator(test_dataframe, batch_size_),
                 validation_steps = len(test_dataframe) // batch_size_,
                 shuffle=True)

Epoch 1/5
Found 46852992 validated image filenames belonging to 2 classes.
Found 46852992 validated image filenames belonging to 2 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 488/3125 [===>..........................] - ETA: 4:23:20 - loss: 11298747347.4800 - accuracy: 0.5255

KeyboardInterrupt: 

In [ ]:
16/3125 [..............................] - ETA: 57:18:16 - loss: 344611778060.7360 - accuracy: 0.5098
69/3125 [..............................] - ETA: 15:37:19 - loss:  79909981066.2464 - accuracy: 0.5095
131/3125 [>.............................] - ETA: 9:37:39 - loss:  42089990112.0998 - accuracy: 0.5041
239/3125 [=>............................] - ETA: 6:30:41 - loss:  23070245628.4792 - accuracy: 0.5109
367/3125 [==>...........................] - ETA: 5:06:25 - loss:  15023947425.7518 - accuracy: 0.5221
487/3125 [===>..........................] - ETA: 4:23:38 - loss:  11321948060.7186 - accuracy: 0.5253

In [ ]:
pred = model.predict_generator(test_generator(test_dataframe, batch_size_), 
                               steps = len(test_dataframe) // batch_size_,
                               verbose=1)